# Clustering 

_Clustering_ refers to a very broad set of techniques for finding subgroups, or clusters, in a dataset. When we cluster the observations of a dataset, we seek to partition them into distinct groups so that the observations within each group are quite similar to each other, while observations in different groups are quite different from each other. For instance, suppose that we have a set of $N$ observations, each with $D$ features. We might define the similarity between two observations as the Euclidean distance between them, which is given by features. 

Both clustering and PCA seek to simplify the data via a small number of summaries, but their mechanisms are different:

- PCA looks to find a low-dimensional representation of the observations that explain a good fraction of the variance;
- Clustering looks to find homogeneous subgroups among the observations.

Since clustering is popular in many fields, there exist a great number of clustering methods. In this section we focus on perhaps the two best-known clustering approaches: $K$-_means clustering_ and _hierarchical clustering_. In $K$-means clustering, we seek to partition the observations into a pre-specified number of clusters. On the other hand, in hierarchical clustering, we do not know in advance how many clusters we want; in fact, we end up with a tree-like visual representation of the observations, called a dendrogram, that allows us to view at once the clusterings obtained for each possible number of clusters, from $1$ to $N$. 

In general, we can cluster observations on the basis of the features in order to identify subgroups among the observations, or we can cluster features on the basis of the observations in order to discover subgroups among the features. In what follows, for simplicity we will discuss clustering observations on the basis of the features, though the converse can be performed by simply transposing the data matrix.

## $K$-means clustering

### $K$-means algorithm

$K$-means clustering is a simple and elegant approach for partitioning a dataset into $K$ distinct, non-overlapping clusters. To perform $K$-means clustering, we must first specify the desired number of clusters $K$; then the $K$-means algorithm will assign each observation to exactly one of the $K$ clusters. 

Watch the 8-minute video below for a visual explanation of $K$-means clustering.

```{admonition} Video

<iframe width="700" height="394" src="https://www.youtube.com/embed/4b5d3muPQmA?start=14" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

[Explaining $K$-Means Clustering, by StatQuest](https://www.youtube.com/embed/4b5d3muPQmA?start=14)

```

The $K$-means clustering procedure results from a simple and intuitive mathematical problem. We begin by defining some notation. Let $C_1, \ldots, C_K$ denote sets containing the indices of the observations in each cluster. These sets satisfy two properties:

1. $C_1 \cup C_2 \cup \ldots \cup C_K = {1, \ldots , N} $. In other words, each observation belongs to at least one of the $K$ clusters.
2. $C_k \cap C_{k′} = \Phi \text{ for all } k \neq k′$. In other words, the clusters are non-overlapping: no observation belongs to more than one cluster.

For instance, if the ith observation is in the $k\text{th}$ cluster, then $i \in C_k $. The idea behind $K$-means clustering is that a good clustering is one for which the within-cluster variation is as small as possible. The within-cluster variation
for cluster $C_k$ is a measure $W(C_k)$ of the amount by which the observations within a cluster differ from each other. Hence we want to solve the problem

```{math}
:label: kmeans-problem
\min_{C_1, \ldots, C_K} \sum_{k=1}^K W(C_k).
```

In words, this formula says that we want to partition the observations into $K$ clusters such that the total within-cluster variation, summed over all $K$ clusters, is as small as possible.

In order to make solving Equation {eq}`kmeans-problem` actionable we need to define the within-cluster variation. There are many possible ways to define this concept, but by far the most common choice involves _squared Euclidean distance_. That is, we define

```{math}
:label: within-cluster-var
W(C_k) = \frac{1}{|C_k|} \sum_{i,j \in C_k} \sum_{d=1}^{D} \left( x_{i,d} - x_{j,d} \right)^2,
```

where $|C_k|$ denotes the number of observations in the $k\text{th}$  cluster. In other words, the within-cluster variation for the kth cluster is the sum of all of the pairwise squared Euclidean distances between the observations in the $k\text{th}$ cluster, divided by the total number of observations in the $k\text{th}$  cluster.

Combining Equations {eq}`kmeans-problem` and {eq}`within-cluster-var`, gives the optimization problem that defines
$K$-means clustering,

```{math}
:label: kmeans-obj
\min_{C_1, \ldots, C_K} \left\{\sum_{k=1}^K \frac{1}{|C_k|} \sum_{i,j \in C_k} \sum_{d=1}^{D} \left( x_{i,d} - x_{j,d} \right)^2\right\}.
```

Now, we would like to find an algorithm to solve Equation {eq}`kmeans-obj`—that is, a method to partition the observations into K clusters such that the objective of Equation {eq}`kmeans-obj` is minimised. This is in fact a very difficult problem to solve precisely, since there are almost $K^N$ ways to partition $N$ observations into $K$ clusters. This is a huge number unless $K$ and $N$ are tiny! Fortunately, a very simple algorithm can be shown to provide a local optimum—a pretty good
solution—to the $K$-means optimisation problem {eq}`kmeans-obj`. This approach is laid out in {prf:ref}`alg:kmeans-algorithm`.


```{prf:algorithm} $K$-means Clustering
:label: alg:kmeans-algorithm

- Randomly assign a number, from $1$ to $K$, to each of the observations. These serve as initial cluster assignments for the observations.

- Iterate until the cluster assignments stop changing:

    - For each of the $K$ clusters, compute the cluster centroid, which is the the vector of the $D$ feature means of the observations in the $k\text{th}$ cluster.

    - Assign each observation to the cluster for which the squared Euclidean distance between the observation and the cluster centroid is smallest.
```


{prf:ref}`alg:kmeans-algorithm` is guaranteed to decrease the value of the objective {eq}`kmeans-obj` at each step. To understand why, the following identity is illuminating:

```{math}
:label: kmeans-obj-identity
\frac{1}{|C_k|} \sum_{i,j \in C_k} \sum_{d=1}^{D} \left( x_{i,d} - x_{j,d} \right)^2 = 2 \sum_{i \in C_k} \sum_{d=1}^{D} \left( x_{i,d} - \bar{x}_{k,d} \right)^2,
```

where $\bar{x}_{k,d}$ is the mean of the $d\text{th}$ feature for the observations in the $k\text{th}$ cluster. In Step 2(a) of {prf:ref}`alg:kmeans-algorithm`, the cluster means for each feature are the constants that minimise the sum-of-squared deviations. In Step 2(b), reallocating the observations can only improve Equation {eq}`kmeans-obj-identity`. This means that as the algorithm is run, the clustering obtained will continually improve until the result no longer changes; the objective of Equation {eq}`kmeans-obj` will never increase. When the result no longer changes, a local optimum has been reached. 

Because the $K$-means algorithm finds a local rather than a global optimum, the results obtained will depend on the initial (random) cluster assignment of each observation in Step 1 of {prf:ref}`alg:kmeans-algorithm`. For this reason, it is important to run the algorithm multiple times from different random initial configurations. Then one selects the best solution, i.e. that for which the objective in Eq. {eq}`kmeans-obj` is the smallest. 

### $K$-means clustering on toy data with `scikit-learn`

Import libraries and generate data:

In [ ]:
import pandas as pd
import numpy as np
from numpy.linalg import svd
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy

%matplotlib inline

In [ ]:
np.random.seed(21)
X = np.random.standard_normal((50, 2))
X[:25, 0] = X[:25, 0] + 3
X[:25, 1] = X[:25, 1] - 4

Fit $K$-means models with $K=2$ and $3$, respectively:

In [ ]:
n_clusters = 2
km1 = KMeans(n_clusters=n_clusters, n_init=20)
km1.fit(X)

n_clusters = 3
km2 = KMeans(n_clusters=n_clusters, n_init=20)
km2.fit(X)

In [ ]:
print(km1.labels_)
print(dir(km1))  # we can use dir to see other saved attributes

The following code cell will plot a figure that shows the results obtained from performing $K$-means clustering on the simulated examples, using three different values of $K$.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.scatter(X[:, 0], X[:, 1], s=40, c=km1.labels_, cmap=plt.cm.prism)
ax1.set_title("$K$-means Clustering Results with K=2")
ax1.scatter(
    km1.cluster_centers_[:, 0],
    km1.cluster_centers_[:, 1],
    marker="+",
    s=100,
    c="k",
    linewidth=2,
)

ax1.scatter(
    X[1, 0],
    X[1, 1],
    alpha=0.8,
    facecolors="none",
    marker="o",
    edgecolors="blue",
    linewidths=3,
)

ax2.scatter(X[:, 0], X[:, 1], s=40, c=km2.labels_, cmap=plt.cm.prism)
ax2.set_title("$K$-means Clustering Results with K=3")
ax2.scatter(
    km2.cluster_centers_[:, 0],
    km2.cluster_centers_[:, 1],
    marker="+",
    s=100,
    c="k",
    linewidth=2,
)


plt.show()

In [ ]:
print("Data sample point : \n", np.around(X[1], 2))

print("Centre coordinates for K=2: \n", np.around(km1.cluster_centers_, 2))

print(
    "Distance of sample point from each cluster centre: \n",
    np.around(km1.transform(X[1].reshape(1, -1))[0], 2),
)

```{admonition} System transparency
:class: important

- When $K=2$, the centres of the two clusters are $[ 3.09 -4.12]$ (red) and $[-0.06  0.51]$ (green). For the data point $ [2.95 -4.11]^{\top} $, its distance to the red cluster centre is $1.48$ and to the green cluster centre is $7.08$. Hence, it is assigned to the red cluster.

- Given a cluster, e.g. the green cluster centred at $[-0.06  0.51]$, any data point $[p, q]$ belong to this cluster should satisfy the following condition: 

\begin{align*}
\begin{aligned}
\sqrt{(p - (-0.06))^2 + (q - 0.51)^2} < & \sqrt{(p-3.09)^2 + (q-(-4.12))^2} \\
0.12 p - 1.02q + 0.2637  < & - 6.18 p + 8.14q + 26.5225 \\
6.3 p - 9.16 q < & 26.2588
\end{aligned} 
\end{align*}


Note: numbers are limited to two decimal places for the sake of clarity.

## Hierarchical clustering

One potential disadvantage of $K$-means clustering is that it requires us to pre-specify the number of clusters $K$. Hierarchical clustering is an alternative approach which does not require that we commit to a particular choice of $K$. Hierarchical clustering has an added advantage over $K$-means clustering in that it results in an attractive tree-based representation of the observations, called a dendrogram.

The code cell below plots a figure to demonstrate _bottom-up_ or _agglomerative_ clustering. This is the most common type of hierarchical clustering, and refers to the fact that a dendrogram (generally depicted as an upside-down tree) is built starting from the leaves and combining clusters up to the trunk. 

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 18))

for linkage, cluster, ax in zip(
    [hierarchy.complete(X), hierarchy.average(X), hierarchy.single(X)],
    ["c1", "c2", "c3"],
    [ax1, ax2, ax3],
):
    cluster = hierarchy.dendrogram(linkage, ax=ax, color_threshold=0)

ax1.set_title("Complete Linkage")
ax2.set_title("Average Linkage")
ax3.set_title("Single Linkage")
plt.show()

In the top panel of figure above, each leaf of the dendrogram represents one of the 50 toy examples generated previously. However, as we move up the tree, some leaves begin to fuse into branches. These correspond to observations that are similar to each other. As we move higher up the tree, branches themselves fuse, either with leaves or other branches. The earlier (lower in the tree) fusions occur, the more similar the groups of observations are to each other. On the other hand, observations that fuse later (near the top of the tree) can be quite different. In fact, this statement can be made precise: for any two observations, we can look for the point in the tree where branches containing those two observations are first fused. The height of this fusion, as measured on the vertical axis, indicates how different the two observations are. Thus, observations that fuse at the very bottom of the tree are quite similar to each other, whereas observations that fuse close to the top of the tree will tend to be quite different.

### The hierarchical clustering Algorithm

The hierarchical clustering dendrogram is obtained via an extremely simple algorithm. We begin by defining some sort of dissimilarity measure between each pair of observations. Most often, Euclidean distance is used. The algorithm proceeds iteratively. Starting out at the bottom of the dendrogram, each of the $N$ observations is treated as its own cluster. The two clusters that are most similar to each other are then fused so that there now are $N$ − 1 clusters. Next the two clusters that are most similar to each other are fused again, so that there now are $N$ − 2 clusters. The algorithm proceeds in this fashion until all of the observations belong to one single cluster, and the dendrogram is complete. {prf:ref}`alg:hierarchical-algorithm` summarises the hierarchical clustering algorithm.

```{prf:algorithm} Hierarchical clustering
:label: alg:hierarchical-algorithm

- Begin with $N$ observations and a measure (such as Euclidean distance) of all the $N(N − 1)/2£ pairwise dissimilarities. Treat each observation as its own cluster.

- For $n = N, N − 1, \ldots, 2$:

    - Examine all pairwise inter-cluster dissimilarities among the $n$ clusters and identify the pair of clusters that are least dissimilar (that is, most similar). Fuse these two clusters. The dissimilarity between these two clusters indicates the height in the dendrogram at which the fusion should be placed.

    - Compute the new pairwise inter-cluster dissimilarities among the $N − 1$ remaining clusters.
```

We have a concept of the dissimilarity between pairs of observations, but how do we define the dissimilarity between two clusters if one or both of the clusters contains multiple observations? The concept of dissimilarity between a pair of observations needs to be extended to a pair of groups of observations. This extension is achieved by developing the notion of linkage, which defines the dissimilarity between two groups of observations. The four most common types of linkage—_complete, average, single,_ and _centroid_. {numref}`linkage-clustering` summarises the four types of linkage. The figure generated in the code cell above show the clustering results of average, complete, and single linkage on the toy dataset, where we can obersve that average and complete linkage tend to yield more balanced clusters.

```{list-table} Types of linkage in hierarchical clustering
:header-rows: 1
:widths: "auto"
:name: linkage-clustering
* - Linkage
  - Description  
* - **Complete**
  - Maximal inter-cluster dissimilarity. Compute all pairwise dissimilarities between the observations in cluster A and the observations in cluster B, and record the largest of these dissimilarities. 
* - **Single**
  - Minimal inter-cluster dissimilarity. Compute all pairwise dissimilarities between the observations in cluster A and the observations in cluster B, and record the smallest of these dissimilarities. Single linkage can result in extended, trailing clusters in which single observations are fused one-at-a-time.
* - **Average**
  - Mean inter-cluster dissimilarity. Compute all pairwise dissimilarities between the observations in cluster A and the observations in cluster B, and record the average of these dissimilarities.
* - **Centroid**
  - Dissimilarity between the centroid for cluster A (a mean vector of length p) and the centroid for cluster B. Centroid linkage can result in undesirable inversions.
```

## Exercises

Clustering the Observations of the NCI60 Data



In [ ]:
X = pd.read_csv("https://github.com/pykale/transparentML/raw/main/data/NCI60_data.csv")
y = pd.read_csv("https://github.com/pykale/transparentML/raw/main/data/NCI60_labs.csv")

In [ ]:
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), index=y.iloc[:, 0], columns=X.columns)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 20))

for linkage, cluster, ax in zip(
    [hierarchy.complete(X_scaled), hierarchy.average(X), hierarchy.single(X_scaled)],
    ["c1", "c2", "c3"],
    [ax1, ax2, ax3],
):
    cluster = hierarchy.dendrogram(
        linkage,
        labels=X_scaled.index,
        orientation="right",
        color_threshold=0,
        leaf_font_size=10,
        ax=ax,
    )

ax1.set_title("Complete Linkage")
ax2.set_title("Average Linkage")
ax3.set_title("Single Linkage")
plt.show()

In [ ]:
plt.figure(figsize=(10, 20))
cut4 = hierarchy.dendrogram(
    hierarchy.complete(X_scaled),
    labels=X_scaled.index,
    orientation="right",
    color_threshold=140,
    leaf_font_size=10,
)
plt.vlines(
    140, 0, plt.gca().yaxis.get_data_interval()[1], colors="r", linestyles="dashed"
)
plt.show()

$K$-means

In [ ]:
np.random.seed(21)
km3 = KMeans(n_clusters=4, n_init=50)
km3.fit(X_scaled)

In [ ]:
km3.labels_

Combine with PCA

In [ ]:
pca = PCA()
X_scaled_ = StandardScaler().fit_transform(X)
df_plot = pd.DataFrame(pca.fit_transform(X_scaled_))

In [ ]:
plt.figure(figsize=(10, 20))
pca_cluster = hierarchy.dendrogram(
    hierarchy.complete(X_scaled),
    labels=X_scaled.index,
    orientation="right",
    color_threshold=100,
    leaf_font_size=10,
)

In [ ]:
# Hierarchy based on Principal Components 1 to 5
plt.figure(figsize=(10, 20))
pca_cluster = hierarchy.dendrogram(
    hierarchy.complete(df_plot.iloc[:, :5]),
    labels=X_scaled.index,
    orientation="right",
    color_threshold=100,
    leaf_font_size=10,
)